<a href="https://colab.research.google.com/github/v-vedantha/Project-8/blob/master/Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pylab inline
from pprint import pprint
import numpy as np

import matplotlib.pyplot as plt

import plotly.express as px
from scipy.stats import cauchy
import plotly.graph_objects as go
from scipy.stats import norm
from iminuit import Minuit, describe
%matplotlib inline
from plotly.subplots import make_subplots
from iminuit.util import make_func_code

Populating the interactive namespace from numpy and matplotlib


In [ ]:
# Downloads the data from files for figure 1
def createF_1():
    
    f = open("DataasCSV.txt", "r")
    f_ = open("errors_fig1.txt", "r")
    content_=f_.read().split()
    content = f.read()
    x = content.split()
    inputs = []
    outputs = []
    errors=[]
    print(len(x), len(content_))
    n = 0
    while n<len(x):
        if(n%2 == 0):
            inputs.append(float(x[n]))
        else:
            outputs.append(float(x[n]))
            errors.append(float(content_[int(n/2)]))

        n += 1
    return (np.asarray(inputs), np.asarray(outputs), np.asarray(errors))

In [ ]:
# Downloads Figure 2 data from files
def createF_2():
    
    f = open("Data2.txt", "r")
    f_ = open("errors_fig2.txt", "r")
    content_=f_.read().split()
    content = f.read()
    x = content.split()
    inputs = []
    outputs = []
    errors=[]
    print(len(x), len(content_))
    n = 0
    while n<len(x):
        if(n%2 == 0):
            inputs.append(float(x[n]))
        else:
            outputs.append(float(x[n]))
            errors.append(float(content_[int(n/2)]))

        n += 1
    return (np.asarray(inputs), np.asarray(outputs), np.asarray(errors))

In [ ]:
# Saves figure 1 and figure 2 data 
# REALIN is the x-values (i.e. the inputs)
# REALOUT is the y-values (i.e. the outputs)
# REALERRORS is the error bars on y for a given x.

REALIN_1, REALOP_1, REALERROR_1 = createF_1()
REALIN_2, REALOP_2, REALERROR_2 = createF_2()

264 132
292 146


In [ ]:
# Represents the energies of the lorentzians for Figure 1
LORENTZIANS_ENERGY_1 = np.array([[ 0,   0.0000,   0.0000,   0.0000,   0.0000],
        [ 19.8,  21.7,  22.8805,  23.639,  0],
        [ 36.3,  38.744,  40.2182,  41.1752,   0.0000],
        [117,   0.0000,   0.0000,   0.0000,   0.0000],
        [255.4,   0.0000,   0.0000,   0.0000,   0.0000]])

# Represents the energies of the lorentzians in Figure 2
LORENTZIANS_ENERGY_2 = np.array([[ 0,   0.0000,   0.0000,   0.0000,   0.0000],
        [ 19.8,  21.7,  22.8805,  23.639,  0],
        [ 36.3,  38.744,  40.2182,  41.1752,   0.0000],
        [117,   0.0000,   0.0000,   0.0000,   0.0000],
        [255.4,   0.0000,   0.0000,   0.0000,   0.0000]])

# Placeholder for another technique we were trying at the time
LOCATION_THREE_HALFS = np.array([[]])

# Stores the magnitudes (intensities) of each lorentzian.
LORENTZIANS_MAGNITUDES = np.array([[100, 0, 0, 0, 0],
                                     [13.05, 2.5, .9, 1.8, 0],
                                     [2, 0.4, 0.1, 0.1, 0],
                                     [1.3, 0, 0, 0, 0],
                                     [.1, 0, 0, 0, 0]])

# Since there is spin orbit splitting, we have to store the spin orbit splitting energies for each lorentzian.
deltas_l=np.array([[ 0,   0.0000,   0.0000,   0.0000,   0.0000],
        [ .69, .69, .69, .69, .69],
        [ 0,0,0,0,   0.0000],
        [1.49,   0.0000,   0.0000,   0.0000,   0.0000],
        [9,   0.0000,   0.0000,   0.0000,   0.0000]])

# Stores spin orbit splitting energies for shakeoffs
deltas_a= np.array([[0], [.69], [0], [1.49], [9]])

# Stores the relative energy of spin orbits for the shakeoffs
factors_half_a = np.array([[0], [1.0], [1.0], [2.0], [1.0]])
factors_lower_a = np.array([[0.0], [2.0], [1.0], [3.0], [2.0]])

# Stores the relative energy of spin orbits for lorentzians
factors_half_l = np.array([[ 1,   0.0000,   0.0000,   0.0000,   0.0000],
        [ 1, 1, 1, 1, 1],
        [ 1.0,1.0,1.0,1.0,   0.0000],
        [2,   0.0000,   0.0000,   0.0000,   0.0000],
        [1*1.02,   0.0000,   0.0000,   0.0000,   0.0000]])
factors_lower_l = np.array([[ 0,   0.0000,   0.0000,   0.0000,   0.0000],
        [ 2,  2, 2, 2, 2],
        [ 0,0,0,0,   0.0000],
        [3,   0.0000,   0.0000,   0.0000,   0.0000],
        [2,   0.0000,   0.0000,   0.0000,   0.0000]])

# Stores the dropoff energies (the x values) for the shakeoffs
DROPOFF_ENERGY = np.array([[0.00001], [26.1], [44.3], [135], [267]])

# Stores a rough estimate of the magnitudes of the dropoffs
# Not necessary, but can provide insights when fitting.
DROPOFF_MAGNITUDES = np.array([[0], [7], [1.3], [3.8], [1.2]])

# Sets up parameters to calculate the lorentzians, including full with half maximums and gammas.
DROPOFF_NUM = 5
FWHM_1 = np.array([[2.7],
                    [2.7],
                    [3.1],
                    [2.77],
                    [3.93]])
GAMMA_1 = FWHM_1/2
FWHM_2 = np.array([[2.7],
                    [2.7],
                    [3.1],
                    [2.77],
                    [4.022]])
GAMMA_2 = FWHM_2/2

# Estimated main energies from paper. We find better fits and update these.
MAIN_ENERGY_1 = 17823.3
MAIN_ENERGY_2 = 900.6

# Increasing NUM_FIXED increases the precision of the convolution, however, it also takes longer to compute.
# Dictates the number of elements ot use in convolution calculation.
NUM_GAUSSIANS = 1
NUM_FIXED=800

# Estimated standard deviations (widths) of core state.
STD_1 = 23.5/2.355
STD_2 = 5.9/2.355

# Scaled heights of the two figures
H_1 = .178
H_2 = 25.7


# Calculates the convolution matrices
G = np.linspace(-2.32, 2.32, NUM_FIXED)
GM=np.asarray(norm.pdf(G))

# Shakeup precalculated variables. Allows you to calculate the spin states more cleanly.
LORENTZIANS_DISTRIBUTIONS_1 = np.zeros((5,1))
LORENTZIANS_DISTRIBUTIONS_MAGNITUDES_1 = np.ones((5,1))
LORENTZIANS_DISTRIBUTIONS_2 = np.zeros((5,1))
LORENTZIANS_DISTRIBUTIONS_MAGNITUDES_2 = np.ones((5,1))


In [ ]:
# Contains all methods relating to calculating the spectra and plotting
class Model():
    def __init__(self, REALIN_1, REALOP_1, REALERROR_1, REALIN_2, REALOP_2, REALERROR_2):
        self.REALIN_1=REALIN_1 
        self.REALOP_1=REALOP_1
        self.REALERROR_1=REALERROR_1
        self.REALIN_2=REALIN_2        
        self.REALOP_2=REALOP_2
        self.REALERROR_2=REALERROR_2
        
        
    # This stores the 6th factor in the shakeoff term
    # The goal of this factor is to reduce the correlation between the shakeoff
    # parameters of intensity and slope.
    # Also must have zero units
    def factor_6(self, EB, DROPOFF_ENERGY_, version, params):
        
        # The function we came up with for the best correlation reduction term
        r = np.log(EB.flatten()[:5]/DROPOFF_ENERGY_.flatten())
        if version == True:
            return r.reshape(1,1,-1,1)
        else:
            # An older version of the function above
            params[:5] = params[:5]/np.log(EB.flatten()[:5])/r
            print(params)
            return params
        
    # The shakeoff term consists of five factors which we split up
    def get_factors(self, n_prime, intratio, energy, MAIN_ENERGY, drop, EB):
        
        #
        # The first four factors are part of the levinger function from the cited paper
        # The dropoff factor ensures that the levinger function is cutoff without making it a discontinuous function
        # The fifth factor is a normalizing factor which undoes the relationship between E_B and the intensity.
        # The sixth factor is the factor 6 above. We were testing out systems without it
        # Returns the levinger functions for all 5 dropoffs
        # Shape : N, 50, 5, 50
        #
        
        factor_1 = np.power(1-np.exp(-2*3.141592 * n_prime),-1)
        
        factor_2 = np.power(n_prime,8)
        
        #
        # We do not need to calculate this value, because of the fix that Dr. Robertson found (in paper).
        # Essentially, he found a simplification for the shakeoff amplitudes that works well, which removes 
        # this somewhat arbitrary technique to cut off the shakeoffs at their peak (with convolution)
        #
        #dropoff_factor = (np.tanh(99999 * (energy.reshape(-1,NUM_FIXED, DROPOFF_NUM*2, NUM_GAUSSIANS) - (MAIN_ENERGY-drop)))+1) * 900 + 1
        factor_3 = np.power(np.power(n_prime, 2)+1, -4)
        
        factor_4 = np.exp(-4*n_prime*np.arctan(1/n_prime))
        
        factor_5 = 1/np.power(EB.reshape(1,1,-1,1), .5)
        
        # Returns the product of those factors.
        return factor_1*factor_2*factor_3*factor_4*intratio*np.concatenate((DROPOFF_MAGNITUDES.reshape(1,1,-1,1), DROPOFF_MAGNITUDES.reshape(1,1,-1,1)), axis=2) * factor_5# * factor_6

    def amplitude(self, energy, whichPlot, relative_magnitudes_a, relative_magnitudes_l, X_Shift, E_B,  H1, H2, ME1, ME2):
        
        # Decides which plot we are using
        if whichPlot == 1:
            MAIN_ENERGY = ME1
            relative_magnitudes_ = relative_magnitudes_a*H1
        if whichPlot == 2:
            relative_magnitudes_ = relative_magnitudes_a*H2
            MAIN_ENERGY = ME2


        # Shifts the entire spectrum according to the energy of the core
        DROPOFF_ENERGY_ = DROPOFF_ENERGY+X_Shift
        
        
        # We concenate both spin states to track them easier.
        # Drop stores the dropoff energy (the peak in the unconvolved spectrum) of each shakeoff
        drop = np.concatenate((DROPOFF_ENERGY_.reshape(1,1,-1,1) + deltas_a.reshape(1,1,-1,1), DROPOFF_ENERGY_.reshape(1,1,-1,1)), axis=2)

        # W stores the energy of each shakeoff.
        W = MAIN_ENERGY - drop - energy.reshape(-1,NUM_FIXED,DROPOFF_NUM*2, NUM_GAUSSIANS)

        # Calculates n' from the levinger paper.
        n_prime = np.sqrt(np.concatenate((E_B.reshape(1,1,-1, 1), E_B.reshape(1,1,-1,1)), axis=2)/np.abs(W+1e-4))

        
        # Each spin state has its own scaling factor, stored here.
        factors_final = np.concatenate((factors_half_a, factors_lower_a))
        
        # Generates the convolved spectrum with the factors for the spin states above.
        spectrum = self.get_factors(n_prime, factors_final, energy, MAIN_ENERGY,drop, np.concatenate((E_B.reshape(1,1,-1, 1), E_B.reshape(1,1,-1,1)), axis=2))*np.concatenate((relative_magnitudes_.reshape(1,1,-1,1), relative_magnitudes_.reshape(1,1,-1,1)), axis=2)
        
        # The fix we've applied to smoothen the peak more simply. Noted in paper.
        spectrum_convolved = spectrum * (.25 + 1/6.283184*np.arctan(2*W/np.concatenate((GAMMA_1.reshape(1,1,-1,1), GAMMA_2.reshape(1,1,-1,1)), axis=2)))
        
        # Return the sum of the two spin states.
        return spectrum_convolved
        
    
    def lorentzians(self, energy, whichPlot, relative_magnitudes_a, relative_magnitudes_l, X_Shift, E_B,H1, H2, ME1, ME2):

        # Decides which figure we are using
        if whichPlot == 1:
            MAIN_ENERGY = ME1
            LORENTZIANS_ENERGY = LORENTZIANS_ENERGY_1+X_Shift
            relative_magnitudes_ = relative_magnitudes_l*H1            
        if whichPlot == 2:
            LORENTZIANS_ENERGY=LORENTZIANS_ENERGY_2+X_Shift
            MAIN_ENERGY = ME2
            relative_magnitudes_ = relative_magnitudes_l*H2

        # Creates all the lorentzians using the default lorentzian formula with the applied heights and widths for the higher energy spin state (bottom comes from the fact that it appears below that spin state on the plot)
        bottom = (energy.reshape(-1,NUM_FIXED,1,1)-(MAIN_ENERGY - LORENTZIANS_ENERGY.reshape(1,1,DROPOFF_NUM,-1)-deltas_l.reshape(1,1,DROPOFF_NUM,-1)))**2 + GAMMA_1.reshape(1,1,DROPOFF_NUM, 1)**2
        
        # Scales the lorentzian according to the scaling parameters we train and the default spin state parameters.
        scaling_factor = relative_magnitudes_.reshape(1,1,-1,1) * LORENTZIANS_MAGNITUDES.reshape(1,1,DROPOFF_NUM,-1) * GAMMA_1.reshape(1,1,DROPOFF_NUM,1)**2
        
        # Repeats above technique for the higher lorentzians.
        bottom_2 = (energy.reshape(-1,NUM_FIXED,1,1)-(MAIN_ENERGY - LORENTZIANS_ENERGY.reshape(1,1,DROPOFF_NUM,-1)))**2 + GAMMA_2.reshape(1,1,DROPOFF_NUM, 1)**2
        scaling_factor_2 = relative_magnitudes_.reshape(1,1,-1,1) * LORENTZIANS_MAGNITUDES.reshape(1,1,DROPOFF_NUM,-1) * GAMMA_2.reshape(1,1,DROPOFF_NUM,1)**2
        
        # Combines the two lorentzians, but does not sum them up because we need to calculate their convolutions separately
        return np.concatenate((scaling_factor * factors_half_l.reshape(1,1,DROPOFF_NUM,-1)/bottom, scaling_factor_2 * factors_lower_l.reshape(1,1,DROPOFF_NUM,-1)/bottom_2), axis=3)

    # Generates the spectrum of shakeups and shakeoffs and applies appropriate convolution to them.
    def gaussianify(self, energy, whichPlot, useGaussian, useLorentzian, useAmplitude, relative_magnitudes_a, relative_magnitudes_l, X_Shift, E_B , H1, H2, ME1, ME2, STD_1, STD_2, lorentzian=1):
        
        # useX represents wether we use that term in our calculation. Lorentzians=Shakeup, Amplitude=Shakeoff, Gaussian=Gaussian convolution
        # Fit the height, ME, and the width by using about 2 standard deviations 
        # Fit the rest keeping the height, ME, and width constant
        if whichPlot == 1:
            
            GAUSSIAN_MAGNITUDES = GM/STD_1
            GAUSSIAN = G*STD_1
            WIDTH_1 = GAUSSIAN[1] - GAUSSIAN[0]
            GAUSSIAN_MAGNITUDES *= WIDTH_1
        if whichPlot == 2:
            
            GAUSSIAN_MAGNITUDES = GM/STD_2
            GAUSSIAN = G*STD_2
            WIDTH_2 = GAUSSIAN[1] - GAUSSIAN[0]
            GAUSSIAN_MAGNITUDES *= WIDTH_2

        # This is the energy input (the x's) scaled appropriately.
        inner = useGaussian*GAUSSIAN.reshape(1,-1,1,1) + energy.reshape(-1,1,1,1) + 1*np.concatenate((LORENTZIANS_DISTRIBUTIONS_1.reshape(1,1,DROPOFF_NUM,NUM_GAUSSIANS), LORENTZIANS_DISTRIBUTIONS_2.reshape(1,1,DROPOFF_NUM,NUM_GAUSSIANS)), axis=2)

        # This is the levinger functions when run on the inputs
        mid_amp = self.amplitude(inner, whichPlot, relative_magnitudes_a, relative_magnitudes_l, X_Shift, E_B, H1, H2, ME1, ME2)
        
        # This is the levinger functions summed up with lorentzian convolution
        amp_1 = np.sum(mid_amp* np.concatenate((LORENTZIANS_DISTRIBUTIONS_MAGNITUDES_1.reshape(1,1,DROPOFF_NUM,NUM_GAUSSIANS), LORENTZIANS_DISTRIBUTIONS_MAGNITUDES_2.reshape(1,1,DROPOFF_NUM,NUM_GAUSSIANS)), axis=2), (2,3))
        
        # This is the gaussian convolution applied to the levinger functions post-convolution
        amp = np.sum(amp_1 * GAUSSIAN_MAGNITUDES.reshape(1, -1), 1)
        
        # This is the gaussian convolution applied to the lorentzian
        lor = self.lorentzians(useGaussian*GAUSSIAN.reshape(1,-1) + energy.reshape(-1,1), whichPlot, relative_magnitudes_a, relative_magnitudes_l, X_Shift, E_B, H1, H2, ME1, ME2) * GAUSSIAN_MAGNITUDES.reshape(1,-1,1,1)
        
        # Summing up the lorentzians
        lor = np.sum(lor, (1,2,3))
        
        # Returns the final spectrum
        return useAmplitude*amp + useLorentzian*lor
    
    # Plots the spectrum.
    def plot(self, whichPlot, min, max, par, label, lower, height, l=1,plotG=False, plotHG=False, plotF=False, plotDif=False, useLorentzian=1, useAmplitude=1):
        # Whichplot is the figure you want to plot
        # min, max are the ranges of the figure
        # par is the parameters
        # label is the y axis labels
        # height is the height of the grey line representing the core-shakeoff to rest of the graph transition

        # Decides on which plot to use
        if whichPlot == 1:
            REALIN = REALIN_1
            REALOP = REALOP_1
            REALERROR=REALERROR_1
        else:
            REALIN=REALIN_2
            REALOP=REALOP_2
            REALERROR=REALERROR_2
            
        # 
        # Plots only in a certain range identified by min and max
        #
        fs=35
        curr_min = -1
        curr_max = 200000
        index_min = 0
        index_max = 0
        for i in REALIN:

            if i < min:
                index_min+=1

            if i < max:
                index_max+=1

        # Creates the figure we will plot on.
        fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0, row_heights=[66,33])
        fig.update_layout(template="simple_white", font=dict(family="Times New Roman"size= fs,color="#000000"), showlegend=False)
        fig.update_yaxes(title_text=label, row=1, linewidth=2, col=1, mirror=True, ticks='outside', showline=True)
        fig.update_yaxes(title_text="Residuals", row=2, col=1, mirror=True,linewidth=2, ticks='outside', showline=True)
        fig.update_xaxes(title_text="Electron Energy (eV)", row=2, col=1, linewidth=2, tickformat=".,")
        fig.update_xaxes(row=1, col=1, mirror=True, ticks='outside',linewidth=2,  showline=True)
        fig.update_layout(autosize=False,width=1000,height=1000)
        fig.add_shape(go.layout.Shape(type="line",x0=REALIN[index_min],y0=0,x1=REALIN[index_max-1],y1=0,line=dict(color="Black",width=2,dash="dot",)), row=1,col=1)
        fig.add_shape(go.layout.Shape(type="line",x0=lower,y0=-4,x1=lower,y1=4,line=dictcolor="Black",width=3)), row=2, col=1)
        fig.add_shape(go.layout.Shape(type="line",x0=lower,y0=0,x1=lower,y1=height,line=dict(color="Black",width=3), row=1, col=1)
        fig.add_shape(go.layout.Shape(type="line",x0=REALIN[index_min],y0=0,x1=REALIN[index_max-1], y1=0, line=dict(color="Black",width=2,dash="dot",)), row=2,col=1)
        
        # HG is the convolved version of the spectrum
        hg = self.gaussianify(REALIN, whichPlot, 1, useLorentzian, 1, par[0:5], par[5:10], par[10:15].reshape(5,1), par[15:20], par[20], par[21], par[22], par[23], par[24], par[25], lorentzian=l)
        
        # Plots the graph with logn scaling on y axis to better see structure.
        if plotG:
            
            # Store the energies but with a lot more points plotted to see finer detail
            rr=np.linspace(17400, 17900, 700)
            
            # Calculates the scaled graph. g, g2 are used so the y-axis scalin has numbers close to 1
            g = self.gaussianify(rr, whichPlot, 0, 0, 1, par[0:5], par[5:10], par[10:15].reshape(5,1), par[15:20], par[20], par[21], par[22], par[23], par[24], par[25], lorentzian=l)
            g2=self.gaussianify(rr, whichPlot, 0, 1, 1, par[0:5], par[5:10], par[10:15].reshape(5,1), par[15:20], par[20], par[21], par[22], par[23], par[24], par[25], lorentzian=l)
            s = np.sum(g2)
            g=g/s
            g2=g2/s

            
            # Creates the graph
            fig = make_subplots(rows=1, cols=1, shared_xaxes=True, vertical_spacing=0)
            fig.update_layout(template="simple_white", font=dict(family="Times New Roman",size= fs,color="#000000"), showlegend=False)
            fig.update_yaxes(title_text="Intensity, %", row=1, linewidth=2, col=1, mirror=True,ticks='outside',showline=True)
            fig.update_xaxes(title_text="Electron Energy (eV)", row=2, col=1, linewidth=2, tickformat=".,")
            fig.update_xaxes(row=1, col=1, mirror=True, ticks='outside',linewidth=2,  showline=True)
            fig.update_layout(autosize=False,width=1000,height=1000)
            fig.add_trace(go.Scatter(x=rr, y=g,mode='lines'))
            fig.add_trace(go.Scatter(x=rr, y=g2,mode='lines'))

        # Plots the graph without scaling.
        if plotDif:
            fig.add_trace(go.Scatter(x=np.asarray(REALIN[index_min:index_max]), y=np.asarray((-hg[index_min:index_max] + REALOP[index_min:index_max])/REALERROR[index_min:index_max]),
                                mode='markers'), row=2, col=1)
            fig.add_trace(go.Scatter(x=np.asarray(REALIN[index_min:index_max]), y=np.asarray(hg[index_min:index_max]), line=dict(color='royalblue'),
                                mode='lines'))
            fig.add_trace(go.Scatter(x=np.asarray(REALIN[index_min:index_max]), y=np.asarray(REALOP[index_min:index_max]), error_y=dict(
                                type='data', # value of error bar given in data coordinates
                                array=np.asarray(REALERROR[index_min:index_max]),
                                visible=True),
                                mode='markers'))
        fig.show()
        
    # Calculates the squared error loss for iminuit to fit on
    def loss(self, par_):  
        
        # We're taking iminuit parameters and converting them into a numpy array to make it easier to work with 
        par = np.asarray(par_)

        # Calculates the convolved spectrums for both figures
        hg_1 = self.gaussianify(REALIN_1, 1, 1, 1, 1, par[0:5],  par[5:10], par[10:15].reshape(5,1) ,par[15:20], par[20], par[21], par[22], par[23], par[24], par[25])
        hg_2 = self.gaussianify(REALIN_2, 2, 1, 1, 1, par[0:5],  par[5:10], par[10:15].reshape(5,1) ,par[15:20], par[20], par[21], par[22], par[23], par[24], par[25])
        
        # Computes the chi square value, multiplying factors add weightings.
        # The commented out bounds are the bounds for different regions. Core, Shakeoffs, and so on. Only used to make it easier to switch between them when testing.
        loss_1 = np.power((REALOP_1-hg_1)/(REALERROR_1),2) [:90]#[:90]#[96:125]#[:-11]#[96:125]#[:90]#
        loss_2 = np.power((REALOP_2-hg_2)/(REALERROR_2),2) [:101]#[:101]#[110:135]#[:-5]#[101:120]#[:103]
    
        # The chi square loss is summed for both figures
        loss=np.sum(loss_1) +np.sum(loss_2)

        # Print out to see progress, and store information of notebook crashes
        print(loss)
        print(repr(par))
        return loss
    
    # Gets intensity of a particular state.
    def get_intensities(self, par_, whichPlot, REALIN):
        
        # When specifying inputs, set all variables corresponging to energies to 0 except the one whose intensity you are trying to calculate
        par = np.asarray(par_)
        hg = self.gaussianify(REALIN, whichPlot, 1, 1, 1, par[0:5], par[5:10], par[10:15].reshape(5,1), par[15:20], par[20], par[21], par[22], par[23], par[24], par[25],  lorentzian=1)
        return np.sum(hg)
    

In [ ]:
# Initialize model
M = Model(REALIN_1, REALOP_1, REALERROR_1, REALIN_2, REALOP_2, REALERROR_2)

# Core state "hyperparameters"
# Same as above, but moved down for easier editing.
STD_1 = 23.5/2.355
STD_2 = 5.9/2.355
MAIN_ENERGY_1 = 17823.3
MAIN_ENERGY_2 = 900.6
H_1 = .178
H_2 = 25.7

Contains example values.


In [ ]:
# Example local minimum. May not be fully up to date.
m_binned = Minuit.from_array_func(M.loss, (4.34294482e+00,  2.65537599e+01,  6.92902835e+01,  1.41800375e+00,
        1.70465972e+00,  1.04891547e+00,  5.26598850e-01,  8.23968722e-01,
        9.20775549e-01,  3.84986853e-01,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  9.70000000e+00, -1.68140372e+01,  1.00000000e+01,
        9.11478559e+00,  3.82400862e+01,  1.93461207e+02,  3.03461383e+03,
        1.79129666e-01,  2.57777795e+01,  1.78226541e+04,  9.00499209e+02,
        9.85870946e+00,  2.62788239e+00), pedantic=False, error_x12=0.1, limit_x0=(0,None),limit_x1=(0,None), 
                                  limit_x2=(0,None), limit_x3=(0,None), limit_x4=(0,None), limit_x5=(0,None), limit_x6=(0,None), limit_x7=(0,None), fix_x11=True, fix_x0=True, fix_x15=True,
                                  limit_x8=(0,None), limit_x15=(0,None), limit_x16=(0,None), limit_x17=(0,None), limit_x18=(0,None), limit_x19=(0,None), fix_x20 = True, fix_x21=True, fix_x22=True, fix_x23=True, fix_x24=True, fix_x25=True,
                                  limit_x9=(0,None), fix_x5=True, fix_x10=True, limit_x11=(-1,1), errordef=1)


# Run this to perform the fits using iminuit.
c=m_binned.migrad()

' fix_x20 = True, fix_x21=True, fix_x22=True, fix_x23=True, fix_x24=True, fix_x25=True,'

Methods to store the fit values and errors/correlations.

In [ ]:
def write(location, array):
    f=open('storedvalues/' + location + '.txt', "w")
    st = np.array2string(array).replace(' ', '\t')
    st=st.replace('\t\t', '\t')
    st=st.replace('\n\t', '\t')
    st=st.replace('[', '\n[')
    st=st.replace('\t\t', '\t')
    f.write(st)
    f.close()


In [ ]:
write("214_values", m_binned.np_values())

In [ ]:
write("fit_final_ errors", m_binned.np_errors())

In [ ]:
write("fit1 correlations", np.asarray(m_binned.matrix(correlation=True)))

In [ ]:
write('full fit end v2', m_binned.np_values()[-6:])

In [ ]:
write('full fit errors v2', m_binned.np_errors()[-6:])